In [0]:
import pandas as pd
import json
from pandas.io.json import json_normalize  
from google.colab import drive
import datetime
drive.mount('/content/gdrive/',force_remount = True)

Mounted at /content/gdrive/


In [0]:
def roundtime(time):
  if((time%100)<30):
    return time - (time%100)
  else:
    return time + (100 - (time%100))

def arrivaldate(atime,dtime,date):
  if(atime<dtime):
    date += datetime.timedelta(days=1)
    sdate = date.strftime('%Y-%m-%d')
    return sdate
  else:
    return date.strftime('%Y-%m-%d')

In [0]:
#"gdrive/My Drive/Colab Notebooks/weather/ATL/2016-1.json"
weatherdir = "gdrive/My Drive/Colab Notebooks/weather/"
airports = ['ATL','CLT','DEN','DFW','EWR','IAH','JFK','LAS','LAX','MCO','MIA','ORD','PHX','SEA','SFO']

w_df = pd.DataFrame()

for x in airports:
  for i in range(2016,2018):
    for j in range(1,13):
      dir = weatherdir + x + '/' + str(i) + '-' + str(j) + ".json"
      with open(dir, 'r') as f:
        d = json.load(f)

      temp = json_normalize(d['data']['weather'],  'hourly', 'date')
      temp['airport'] = x

      w_df = pd.concat([w_df, temp],ignore_index=True)

w_df['itime'] = w_df.time.astype('int64')
w_df = w_df.drop(['time','weatherIconUrl','FeelsLikeF','winddir16Point','FeelsLikeC','windspeedMiles','DewPointF','HeatIndexF','HeatIndexC','weatherIconUrl','WindGustMiles','weatherDesc','weatherCode','tempF','WindChillF'],axis=1)


In [0]:

w_df.to_csv("weatherdf.csv")
!cp weatherdf.csv "gdrive/My Drive/Colab Notebooks/"
#files.download('weatherdf.csv')

In [0]:
#gdrive/My Drive/Colab Notebooks/flight/2016/On_Time_On_Time_Performance_2016_1/On_Time_On_Time_Performance_2016_1.csv
flightdir = 'gdrive/My Drive/Colab Notebooks/flight/'

f_df = pd.DataFrame()

for i in range(2016,2018):
  for j in range(1,13):
    dir = flightdir + str(i) + '/' + 'On_Time_On_Time_Performance_' + str(i) + '_' + str(j) + '/' + 'On_Time_On_Time_Performance_' + str(i) + '_' + str(j) + '.csv'
    temp = pd.read_csv(dir)
    f_df = pd.concat([f_df, temp],ignore_index=True)

f_df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (77) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (48,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (48,77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/l

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,...,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2016,1,1,6,3,2016-01-06,AA,19805,AA,N4YBAA,43,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1100,1057.0,-3.0,0.0,0.0,-1.0,1100-1159,15.0,1112.0,1424.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,1,1,7,4,2016-01-07,AA,19805,AA,N434AA,43,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1100,1056.0,-4.0,0.0,0.0,-1.0,1100-1159,14.0,1110.0,1416.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,1,1,8,5,2016-01-08,AA,19805,AA,N541AA,43,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1100,1055.0,-5.0,0.0,0.0,-1.0,1100-1159,21.0,1116.0,1431.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,1,1,9,6,2016-01-09,AA,19805,AA,N489AA,43,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1100,1102.0,2.0,2.0,0.0,0.0,1100-1159,13.0,1115.0,1424.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,1,1,10,7,2016-01-10,AA,19805,AA,N439AA,43,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1100,1240.0,100.0,100.0,1.0,6.0,1100-1159,20.0,1300.0,1617.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
f_df.drop(f_df.columns.difference(['FlightDate','Quarter','Year','Month','DayofMonth','DepTime','DepDel15','CRSDepTime','DepDelayMinutes','Origin','Dest','ArrTime','CRSArrTime','ArrDel15','ArrDelayMinutes']), 1, inplace=True)
f_df['round_Dtime'] = f_df.apply(lambda x: roundtime(x['CRSDepTime']),axis=1)
f_df['round_Atime'] = f_df.apply(lambda x: roundtime(x['CRSArrTime']),axis=1)
f_df['A_date'] = f_df.apply(lambda x: arrivaldate(x['CRSArrTime'],x['CRSDepTime'],datetime.datetime.strptime(x['FlightDate'], '%Y-%m-%d')),axis=1)

f_df.head()

,Year,Quarter,Month,DayofMonth,FlightDate,Origin,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15,round_Dtime,round_Atime,A_date
0,2016,1,1,6,2016-01-06,DFW,DTW,1100,1057.0,0.0,0.0,1438,1432.0,0.0,0.0,1100,1500,2016-01-06
1,2016,1,1,7,2016-01-07,DFW,DTW,1100,1056.0,0.0,0.0,1438,1426.0,0.0,0.0,1100,1500,2016-01-07
2,2016,1,1,8,2016-01-08,DFW,DTW,1100,1055.0,0.0,0.0,1438,1445.0,7.0,0.0,1100,1500,2016-01-08
3,2016,1,1,9,2016-01-09,DFW,DTW,1100,1102.0,2.0,0.0,1438,1433.0,0.0,0.0,1100,1500,2016-01-09
4,2016,1,1,10,2016-01-10,DFW,DTW,1100,1240.0,100.0,1.0,1438,1631.0,113.0,1.0,1100,1500,2016-01-10


In [0]:
f_df['A_date'] = f_df.apply(lambda x: arrivaldate(x['CRSArrTime'],x['CRSDepTime'],datetime.datetime.strptime(x['FlightDate'], '%Y-%m-%d')),axis=1)

f_df.head()

,Year,Quarter,Month,DayofMonth,FlightDate,Origin,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15,round_Dtime,round_Atime,A_date
0,2016,1,1,6,2016-01-06,DFW,DTW,1100,1057.0,0.0,0.0,1438,1432.0,0.0,0.0,1100,1500,2016-01-06
1,2016,1,1,7,2016-01-07,DFW,DTW,1100,1056.0,0.0,0.0,1438,1426.0,0.0,0.0,1100,1500,2016-01-07
2,2016,1,1,8,2016-01-08,DFW,DTW,1100,1055.0,0.0,0.0,1438,1445.0,7.0,0.0,1100,1500,2016-01-08
3,2016,1,1,9,2016-01-09,DFW,DTW,1100,1102.0,2.0,0.0,1438,1433.0,0.0,0.0,1100,1500,2016-01-09
4,2016,1,1,10,2016-01-10,DFW,DTW,1100,1240.0,100.0,1.0,1438,1631.0,113.0,1.0,1100,1500,2016-01-10


In [0]:
#f_df.to_csv('flightdf.csv')
#from google.colab import files
#files.download('flightdf.csv')

f_df.to_csv("flightdf.csv")
!cp flightdf.csv "gdrive/My Drive/Colab Notebooks/"

In [0]:
merged = pd.merge(w_df, f_df,left_on = ['airport','itime','date'], right_on = ['Origin','round_Dtime','FlightDate'],how='right')


In [0]:
merged = pd.merge(w_df, merged,left_on = ['airport','itime','date'], right_on = ['Dest','round_Atime','FlightDate'],how='right',suffixes=('_A','_D'))
merged.head()

,windspeedKmph_A,DewPointC_A,cloudcover_A,precipMM_A,pressure_A,WindGustKmph_A,visibility_A,tempC_A,WindChillC_A,winddirDegree_A,humidity_A,date_A,airport_A,itime_A,windspeedKmph_D,DewPointC_D,cloudcover_D,precipMM_D,pressure_D,WindGustKmph_D,visibility_D,tempC_D,WindChillC_D,winddirDegree_D,humidity_D,date_D,airport_D,itime_D,Year,Quarter,Month,DayofMonth,FlightDate,Origin,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15,round_Dtime,round_Atime,A_date
0,11,8,100,0.1,1023,17,10,10,8,318,91,2016-01-01,ATL,0.0,9,-11,26,0.0,1038,19,7,-10,-15,221,91,2016-01-01,DEN,2000.0,2016,1,1,1,2016-01-01,DEN,ATL,1930,1952.0,22.0,1.0,15,30.0,15.0,1.0,2000,0,2016-01-02
1,11,8,100,0.1,1023,17,10,10,8,318,91,2016-01-01,ATL,0.0,8,3,14,0.0,1034,10,10,7,6,343,73,2016-01-01,DFW,2100.0,2016,1,1,1,2016-01-01,DFW,ATL,2125,2138.0,13.0,0.0,27,24.0,0.0,0.0,2100,0,2016-01-02
2,11,8,100,0.1,1023,17,10,10,8,318,91,2016-01-01,ATL,0.0,17,1,0,0.0,1017,19,10,4,0,266,83,2016-01-01,EWR,2100.0,2016,1,1,1,2016-01-01,EWR,ATL,2126,2139.0,13.0,0.0,1,2357.0,0.0,0.0,2100,0,2016-01-02
3,11,8,100,0.1,1023,17,10,10,8,318,91,2016-01-01,ATL,0.0,18,-1,0,0.0,1021,21,10,8,5,70,54,2016-01-01,SFO,1700.0,2016,1,1,1,2016-01-01,SFO,ATL,1645,1645.0,0.0,0.0,14,2347.0,0.0,0.0,1700,0,2016-01-02
4,13,7,100,0.0,1023,22,10,8,6,317,92,2016-01-01,ATL,100.0,9,-11,26,0.0,1038,19,7,-10,-15,221,91,2016-01-01,DEN,2000.0,2016,1,1,1,2016-01-01,DEN,ATL,2014,2000.0,0.0,0.0,111,47.0,0.0,0.0,2000,100,2016-01-02


In [0]:
merged = merged.dropna(axis = 0, how ='any')

In [0]:
merged.to_csv("merged.csv")
!cp merged.csv "gdrive/My Drive/Colab Notebooks/"

In [0]:
merged.shape

(1780890, 46)